<a href="https://colab.research.google.com/github/semishen/ML100Days/blob/master/Day_024_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
### Q1: 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

### A1: 標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 不同的編碼方式對於線性迴歸的影響很大 (mean of cross val score: 0.662 vs 0)；而對於梯度提升樹的影響很小 (mean of cross val score: 0.777 vs 0.793)

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
print('train shape: ', df_train.shape) 
print('test shape: ', df_test.shape)
df.head()

train shape:  (891, 10)
test shape:  (418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
#只取類別值 (object) 型欄位, 存於 object_features 中
# df = df.loc[:, df.dtypes == 'object']

object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
### Q2:  鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)
### A2:  Logistic Regression 在 One Hot Encoder 的 mean of cross val score: 0.801 略優於 Label Encoder 的 mean of cross val score: 0.780，不過運算時間卻多了 5 倍 (0.606 s vs 0.120 s)

In [24]:
# 標籤編碼 + 羅吉斯迴歸
df_label = df.copy()
for col in df_label.columns:
  df_label[col] = LabelEncoder().fit_transform(df_label[col])

#df_label.head()
train = df_label[:train_num]
print('train shape: ', train.shape)

start = time.time()
estimator = LogisticRegression()
score = cross_val_score(estimator, train, train_Y, cv=5).mean()
print('score: ', score)
print('time: ', time.time() - start)

train shape:  (891, 5)
score:  0.7800138095537004
time:  0.11961007118225098


In [25]:
# 獨熱編碼 + 羅吉斯迴歸
df_onehot = df.copy()
df_onehot = pd.get_dummies(df_onehot)

train = df_onehot[:train_num]
print('train shape: ', train.shape)# one-hot encoding 'Name' make a lot of meaningless cols 

start = time.time()
estimator = LogisticRegression()
score = cross_val_score(estimator, train, train_Y, cv=5).mean()
print('score: ', score)
print('time: ', time.time() - start)

train shape:  (891, 2429)
score:  0.8013307388111229
time:  0.6065337657928467
